<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install numpy pandas matplotlib Cython bs4 urllib3 marshmallow pandas_ta scikit-learn tensorflow plotly nbformat prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=2df78ae7042e2fee78f31af6a48cdbde05d1886dc25ad504c66d19d871a0a3cc
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
import requests
import pandas as pd
import math
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from bs4 import BeautifulSoup

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:
    root_url = "https://www.tefas.gov.tr"
    info_endpoint = "/api/DB/BindHistoryInfo"
    concurrently = False
    use_Proxy = False
    fon_type = "YAT"
    proxies = None

    @staticmethod
    def get_combobox_items(url, select_id):
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')
        select_element = soup.find('select', id=select_id)

        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        options = select_element.find_all('option')
        options = list(filter(None, options))

        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')

        return items

    def fetch_info(self, fonunvantip, start_date_initial, end_date_initial):
        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
            counter = range_date.days / range_interval
            counter = math.ceil(counter)
            end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
            counter -= 1

            data = {
                    "fontip": self.fon_type,
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

            info = self._do_post(data)
            info = info_schema.load(info)
            info = pd.DataFrame(info, columns=info_schema.fields.keys())
            info['fonunvantip'] = fonunvantip
            if not info.empty :
                info_result = pd.concat([info_result, info])
                info_result = info_result.reset_index(drop=True)
                info = info.reset_index(drop=True)

            if counter > 0 :
                start_date = end_date + timedelta(days=1)
                end_date = end_date + timedelta(days=range_interval)
                if end_date > end_date_initial :
                    end_date = end_date_initial

        return info_result

    def fetch_info_serial(self, fonunvantips, start_date_initial, end_date_initial):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            if not info.empty :
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

        return merged

    def fetch_info_concurrently(self, fonunvantips, start_date_initial, end_date_initial):
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            merged = pd.DataFrame()
            self.concurrently = True
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                print(f"{future} - {len(info)} records added total records: {len(merged)} " )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
        unvantip: bool = False,
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = [""]
        if unvantip :
            fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")

        self.proxies = None

        if self.concurrently :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    # def get_free_proxy(self):
    #     proxy_address = FreeProxy(timeout=1, rand=True, https=True).get()
    #     return proxy_address

    def _do_post(self, data: Dict[str, str]) -> Dict[str, str]:
        timestamp = int(time.time() * 1000)  # Get current timestamp in milliseconds
        headers = {
         "Connection": "keep-alive",
         "Cache-Control": "no-cache",
         "Pragma": "no-cache",
         "X-Requested-With": "XMLHttpRequest",
         "Sec-Fetch-Mode": "cors",
         "Sec-Fetch-Site": "same-origin",
         "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
         "Accept": "application/json, text/javascript, */*; q=0.01",
         "Origin": "https://www.tefas.gov.tr",
         "Referer": f"https://www.tefas.gov.tr/TarihselVeriler.aspx?timestamp={timestamp}" ,
         }

        response = requests.post(
             url=f"{self.root_url}/{self.info_endpoint}",
             data=data,
             proxies=self.proxies,
             headers=headers,
         )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

tefas = tefas_get()

In [3]:
time_delta = 450
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "market_cap", "number_of_shares", "number_of_investors"], unvantip=False)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data.rename(columns={'price': 'close'}, inplace=True)
fetched_data.rename(columns={'code': 'symbol'}, inplace=True)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)
fetched_data['market_cap_per_investors'] = fetched_data['market_cap'] / fetched_data['number_of_investors']
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]
fetched_data = fetched_data.sort_values(['symbol', 'date'])
fetched_data['open'] = fetched_data.groupby('symbol')['close'].shift(1)
fetched_data['high'] = fetched_data[['open', 'close']].max(axis=1)
fetched_data['low'] = fetched_data[['open', 'close']].min(axis=1)
fetched_data = fetched_data.dropna()
fetched_data.to_csv('tefas.csv', encoding='utf-8-sig', index=False)
fetched_data

 - 444599 records added total records: 444599 


,symbol,date,close,market_cap,number_of_shares,number_of_investors,market_cap_per_investors,open,high,low
76770,AAK,2023-06-15,11.308113,88743098.15,7847737.0,528.0,1.680740e+05,11.364256,11.364256,11.308113
75484,AAK,2023-06-16,11.388840,82648500.82,7256973.0,527.0,1.568283e+05,11.308113,11.388840,11.308113
74196,AAK,2023-06-19,11.392882,77670608.92,6817468.0,526.0,1.476628e+05,11.388840,11.392882,11.388840
72906,AAK,2023-06-20,11.211692,72907110.11,6502775.0,524.0,1.391357e+05,11.392882,11.392882,11.211692
71616,AAK,2023-06-21,11.168418,68287112.99,6114305.0,523.0,1.305681e+05,11.211692,11.211692,11.168418
...,...,...,...,...,...,...,...,...,...,...
356227,ZZL,2024-09-02,47.665448,24276918.47,509319.0,15.0,1.618461e+06,47.593184,47.665448,47.593184
354584,ZZL,2024-09-03,47.695117,24292029.48,509319.0,15.0,1.619469e+06,47.665448,47.695117,47.665448
352933,ZZL,2024-09-04,47.832139,24361817.44,509319.0,15.0,1.624121e+06,47.695117,47.832139,47.695117
351281,ZZL,2024-09-05,47.976250,24435215.49,509319.0,15.0,1.629014e+06,47.832139,47.976250,47.832139


In [4]:
import pandas_ta as ta
import pandas as pd
import numpy as np
from pandas_ta.utils import get_offset, verify_series, signals

def calculate_ta(group):
    # Simple Moving Average (SMA)
    SMA = ta.sma(group['close'], length=5)
    group = pd.concat([group, SMA], axis=1)

    # Relative Strength Index (RSI)
    RSI = ta.rsi(group['close'], length=14, append=True)
    group = pd.concat([group, RSI], axis=1)

    BB = ta.bbands(group['close'], length=20, std=2, ddof=0, mamode=None, talib=None, offset=None)
    group = pd.concat([group, BB], axis=1)

    return group

fetched_data=pd.read_csv('tefas.csv')
fetched_data['close'].astype(float,False)
fetched_data['date'] = pd.to_datetime(fetched_data['date'])
fetched_data['year'] = fetched_data['date'].dt.year
fetched_data['week_no'] = fetched_data['date'].dt.isocalendar().week.astype(str).str.zfill(2)
fetched_data['year_week'] = fetched_data['year'].astype(str) +'-'+ fetched_data['week_no'].astype(str)
fetched_data['day_of_week'] = fetched_data['date'].apply(lambda x: x.strftime('%A'))
#idx = fetched_data.groupby(['symbol', 'year_week'])['date'].idxmax()
#max_prices = fetched_data.loc[idx, ['symbol', 'year_week', 'close']]
#max_prices = max_prices.rename(columns={'close': 'price_at_week_close'})
#fetched_data = fetched_data.merge(max_prices, on=['symbol', 'year_week'], how='left')
#fetched_data['qty'] = 100/fetched_data['close']
#fetched_data['valuation_at_week_close'] = fetched_data['price_at_week_close'] * fetched_data['qty']
fetched_data.sort_values(by=['symbol', 'date'], inplace=True)
fetched_data = fetched_data.groupby(['symbol']).apply(calculate_ta)
fetched_data = fetched_data.dropna()
fetched_data.to_csv('tefas_transformed.csv', encoding='utf-8-sig', index=False)
fetched_data

symbol       date      close   market_cap  number_of_shares  \
symbol                                                                      
AAK    19        AAK 2023-07-17  12.945620  88324200.73         6822709.0   
       20        AAK 2023-07-18  13.163630  90103583.75         6844889.0   
       21        AAK 2023-07-19  13.130010  86879466.07         6616862.0   
       22        AAK 2023-07-20  13.397760  89277539.59         6663617.0   
       23        AAK 2023-07-21  13.549324  88858051.02         6558117.0   
...              ...        ...        ...          ...               ...   
ZZL    426601    ZZL 2024-09-02  47.665448  24276918.47          509319.0   
       426602    ZZL 2024-09-03  47.695117  24292029.48          509319.0   
       426603    ZZL 2024-09-04  47.832139  24361817.44          509319.0   
       426604    ZZL 2024-09-05  47.976250  24435215.49          509319.0   
       426605    ZZL 2024-09-06  47.785748  24338189.42          509319.0   

               number_of_investors  market_cap_per_investors       open  \
symbol                                                                    
AAK    19                    540.0              1.635633e+05  12.854900   
       20                    545.0              1.653277e+05  12.945620   
       21                    554.0              1.568221e+05  13.163630   
       22                    556.0              1.605711e+05  13.130010   
       23                    561.0              1.583922e+05  13.397760   
...                            ...                       ...        ...   
ZZL    426601                 15.0              1.618461e+06  47.593184   
       426602                 15.0              1.619469e+06  47.665448   
       426603                 15.0              1.624121e+06  47.695117   
       426604                 15.0              1.629014e+06  47.832139   
       426605                 15.0              1.622546e+06  47.976250   

                    high        low  ...  week_no year_week day_of_week  \
symbol                               ...                                  
AAK    19      12.945620  12.854900  ...       29   2023-29      Monday   
       20      13.163630  12.945620  ...       29   2023-29     Tuesday   
       21      13.163630  13.130010  ...       29   2023-29   Wednesday   
       22      13.397760  13.130010  ...       29   2023-29    Thursday   
       23      13.549324  13.397760  ...       29   2023-29      Friday   
...                  ...        ...  ...      ...       ...         ...   
ZZL    426601  47.665448  47.593184  ...       36   2024-36      Monday   
       426602  47.695117  47.665448  ...       36   2024-36     Tuesday   
       426603  47.832139  47.695117  ...       36   2024-36   Wednesday   
       426604  47.976250  47.832139  ...       36   2024-36    Thursday   
       426605  47.976250  47.785748  ...       36   2024-36      Friday   

                   SMA_5     RSI_14  BBL_20_2.0  BBM_20_2.0  BBU_20_2.0  \
symbol                                                                    
AAK    19      12.769682  88.609599   10.732879   12.011008   13.289136   
       20      12.885753  90.404116   10.774845   12.103783   13.432722   
       21      12.974315  88.099038   10.832843   12.190842   13.548841   
       22      13.098384  90.234567   10.888250   12.291086   13.693921   
       23      13.237269  91.197469   10.994844   12.407967   13.821091   
...                  ...        ...         ...         ...         ...   
ZZL    426601  47.589465  82.526678   45.910351   46.973127   48.035903   
       426602  47.601164  82.873598   46.029839   47.053828   48.077816   
       426603  47.658296  84.412794   46.068307   47.118052   48.167797   
       426604  47.752428  85.852876   46.179382   47.206211   48.233041   
       426605  47.790940  75.873645   46.300428   47.277857   48.255286   

               BBB_20_2.0  BBP_20_2.0  
symbol                                 
AAK 

In [5]:
time_delta = 15
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")
fetched_data_agg = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "fonunvantip", "title"], unvantip=True)
fetched_data_agg.drop_duplicates(subset=['code', 'fonunvantip'], ignore_index=True, inplace=True)
fon_table = fetched_data_agg.pivot_table(index=['title', 'code'], columns='fonunvantip', aggfunc='size', fill_value=0)
fon_table.reset_index(inplace=True)
fon_table = fon_table.replace(0, False)
fon_table = fon_table.replace(1, True)
fon_table.rename(columns={'code': 'symbol'}, inplace=True)
fon_table['symbol_with_title'] = fon_table['symbol'].astype(str) +' - '+ fon_table['title'].astype(str)
fon_table.to_csv('fon_table.csv', encoding='utf-8-sig', index=False)
fon_table

Agresif Değişken - 44 records added total records: 44 
Alternatif - 33 records added total records: 77 
Altın - 330 records added total records: 407 
Atak Dinamik Büyüme Değişken - 11 records added total records: 418 
Borçlanma Araçları - 825 records added total records: 1243 
Çalışanlarına Yönelik - 22 records added total records: 1265 
Çoklu Varlık - 121 records added total records: 1386 
Değişken - 1474 records added total records: 2860 
Dengeli Değişken - 33 records added total records: 2893 
Diğer Fon Sepeti - 22 records added total records: 2915 
Döviz - 5573 records added total records: 8488 
Döviz Cinsinden İhraç (Dolar) - 5001 records added total records: 13489 
Döviz Cinsinden İhraç (Euro) - 550 records added total records: 14039 
Emtia - 88 records added total records: 14127 
Endeks - 176 records added total records: 14303 
Endeks Hisse Senedi - 44 records added total records: 14347 
Eurobond - 99 records added total records: 14446 
Fon Sepeti - 873 records added total recor

fonunvantip,title,symbol,Agresif Değişken,Alternatif,Altın,Atak Dinamik Büyüme Değişken,Borçlanma Araçları,Dengeli Değişken,Değişken,Diğer Fon Sepeti,...,Serbest,Sürdürülebilirlik Fonları,Uzun Vadeli,Yabancı,Yabancı Fon Sepeti,Çalışanlarına Yönelik,Çoklu Varlık,Özel,İştirak,symbol_with_title
0,AK PORTFÖY 0-5 YIL VADELİ SERBEST (DÖVİZ) FON,AJK,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,AJK - AK PORTFÖY 0-5 YIL VADELİ SERBEST (DÖVİZ...
1,AK PORTFÖY 2025 SERBEST (DÖVİZ) FON,YRB,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,YRB - AK PORTFÖY 2025 SERBEST (DÖVİZ) FON
2,AK PORTFÖY 2028 SERBEST (DÖVİZ) FON,SER,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,SER - AK PORTFÖY 2028 SERBEST (DÖVİZ) FON
3,AK PORTFÖY ADOC SERBEST (DÖVİZ) ÖZEL FON,CJF,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,CJF - AK PORTFÖY ADOC SERBEST (DÖVİZ) ÖZEL FON
4,AK PORTFÖY AK SANDIK SERBEST ÖZEL FON,SSO,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,SSO - AK PORTFÖY AK SANDIK SERBEST ÖZEL FON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,İŞ PORTFÖY İHRACATÇI ŞİRKETLER HİSSE SENEDİ (T...,IHT,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,IHT - İŞ PORTFÖY İHRACATÇI ŞİRKETLER HİSSE SEN...
1648,İŞ PORTFÖY İKİNCİ PARA PİYASASI (TL) FONU,IOO,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,IOO - İŞ PORTFÖY İKİNCİ PARA PİYASASI (TL) FONU
1649,İŞ PORTFÖY İKİNCİ SERBEST (TL) FON,ILI,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,ILI - İŞ PORTFÖY İKİNCİ SERBEST (TL) FON
1650,İŞ PORTFÖY İŞ BANKASI İŞTİRAKLERİ ENDEKSİ HİSS...,TI3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,TI3 - İŞ PORTFÖY İŞ BANKASI İŞTİRAKLERİ ENDEKS...


In [9]:
from prophet import Prophet
import pandas as pd
import plotly.graph_objects as go

# Load your data
data_import = pd.read_csv('tefas_transformed.csv')
symbols = data_import['symbol'].unique()
symbols = ['AFT']

for symbol in symbols:
    data = data_import[data_import['symbol'] == symbol]
    data = data[['date', 'close', 'RSI_14', 'SMA_5']].copy()
    data.rename(columns={'date': 'ds', 'close': 'y'}, inplace=True)

    # Create and fit the Prophet model with RSI_14 as a regressor
    model = Prophet()
    #model.add_regressor('RSI_14')
    #model.add_regressor('SMA_5')
    model.fit(data)

    # Create future dates for prediction
    future = model.make_future_dataframe(periods=30)

    # You need to provide RSI_14 values for future dates
    # Here's an example of using the last RSI_14 value for all future dates
    #future['RSI_14'] = data['RSI_14'].iloc[-1]

    # Make predictions
    forecast = model.predict(future)

    # Create the plot
    fig = go.Figure()

    # Add past close values
    fig.add_trace(go.Scatter(x=data['ds'], y=data['y'], mode='lines', name='Past Close'))

    # Add predicted values
    fig.add_trace(go.Scatter(x=forecast['ds'][-30:], y=forecast['yhat'][-30:], mode='lines', name='Predicted Close'))

    # Set layout
    fig.update_layout(title=f'Close Price Prediction for {symbol}', xaxis_title='Date', yaxis_title='Close Price')

    # Show the plot
    fig.show()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvnosd5sp/oov4hfgt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvnosd5sp/rn4qdhl1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36437', 'data', 'file=/tmp/tmpvnosd5sp/oov4hfgt.json', 'init=/tmp/tmpvnosd5sp/rn4qdhl1.json', 'output', 'file=/tmp/tmpvnosd5sp/prophet_modelg4fr1b__/prophet_model-20240906115324.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:53:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:53:24 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/